In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import Imputer

In [2]:
def to_date2(row):
    x=datetime.strptime(row['date_priced'], '%m/%d/%Y %I:%M %p')
    return x
def to_date1(row):
    x=datetime.strptime(row['buid_date'], '%m/%d/%Y %I:%M %p')
    return x
def fun_b_year(row):
    x=row['build_date']
    return x.year

def fun_p_year(row):
    x=row['date_priced']
    return x.year

In [85]:
def filtering(df):
    
    #encode locations
    ser=pd.Series(np.arange(0,df.locations.unique().size),df.locations.unique())
    df.locations.replace(ser,inplace=True)
    
    #encode farm land....
    #print(df.farm_land.unique())
    ser1=pd.Series(np.arange(0,df.farm_land.unique().size),df.farm_land.unique())
    df.farm_land.replace(ser1,inplace=True)
    
    #encode builder
    #print(df.builder.unique())
    ser2=pd.Series(np.arange(0,11),df.builder.unique())
    #print(ser2)
    df.builder.replace(ser2,inplace=True)
    
    df['buid_date']=df.apply(lambda i:to_date1(i),axis=1)
    df['date_priced']=df.apply(lambda i:to_date2(i),axis=1)
    
    #df['build_year']=df.apply(lambda i:fun_b_year(i),axis=1)
    #df['priced_year']=df.apply(lambda i:fun_p_year(i),axis=1)
    
    for i in range (0, df.index.size):
        df.loc[i,'house_age']=(df.loc[i,'date_priced']-df.loc[i,'buid_date']).days
        df.loc[i,'build_year']=(df.loc[i,'buid_date']).year
        df.loc[i,'priced_year']=(df.loc[i,'date_priced']).year
        
    #encode build year
    ser2=pd.Series(np.arange(0,df.build_year.unique().size),df.build_year.unique())
    df.build_year.replace(ser2,inplace=True)
    
    #encode priced year
    ser3=pd.Series(np.arange(0,df.priced_year.unique().size),df.priced_year.unique())
    df.priced_year.replace(ser3,inplace=True)
        
           
        
    #ffill fillna  to curse and bathrooms coz less correlation with price ...
    df['curse'].fillna(method='ffill',inplace=True)
    df['curse'].fillna(method='bfill',inplace=True)
    df['bathrooms'].fillna(method='ffill',inplace=True)
    df['bathrooms'].fillna(method='bfill',inplace=True)
        
    
    #.96 correlation b/w curse and holy_tree...
    #so fillna of both columns with help of each other....
    d1=df[['curse','holy_tree']]
    d1=d1.fillna(-1)
    p_index(d1)

    for i in range (0,d1.index.size):
        if d1.loc[i,'curse']==-1 and d1.loc[i,'holy_tree']!=-1:
            d1.loc[i,'curse']=(d1.loc[i,'holy_tree']+1)%2

    for i in range (0,d1.index.size):
        if d1.loc[i,'curse']!=-1 and d1.loc[i,'holy_tree']==-1:
            d1.loc[i,'holy_tree']=(d1.loc[i,'curse']+1)%2

    d1[d1.curse==-1]=pd.np.nan
    df['curse']=d1['curse']
    df['holy_tree']=d1['holy_tree']
    

    
    #king_blessing,dst_capital and dst_knight are correlated ..so keep only one ..
    #king_visit and garden also same ,so delete 1 of them.
    df.drop(['garden','dst_capital','dst_knight'],inplace=True,axis=1)
    
    #too low correlation of renovation with price so drop it¶
    df.drop('renovation',inplace=True,axis=1)
    
    #print(df)
    
    df.drop('buid_date',inplace=True,axis=1)
    df.drop('date_priced',inplace=True,axis=1)
    df.drop('house_id',axis=1,inplace=True)    
    
    return df
    
pass

In [86]:
def p_index(df):
    print(df.isnull().sum())
    print(df.isnull().values.sum())
    print(df.dropna().index.size)
    return

def p_heatmap(df):  
    c=df.corr()
    plt.figure(figsize=[18,8])
    sns.heatmap(c,annot=True,cmap="Blues")
    return

In [87]:
df=pd.read_csv('csv/train.csv')

In [88]:
df=filtering(df)

curse        0
holy_tree    0
dtype: int64
0
16500


In [89]:
df.head()

,dst_dock,dst_mkt,dst_tower,dst_river,dining_rooms,bedrooms,bathrooms,king_visit,curse,king_blessing,farm_land,locations,holy_tree,builder,price,house_age,build_year,priced_year
0,138.600639,69.486467,14.394903,63.109038,3.0,4.0,2.0,0,0.0,160,0,0,1.0,0,1470912.0,2636.0,0.0,0.0
1,16.254871,34.145238,101.052598,53.249336,2.0,4.0,2.0,1,0.0,133,1,1,1.0,1,1241030.0,4270.0,1.0,1.0
2,6.555428,78.445508,88.480988,72.245533,3.0,NaN,2.0,0,0.0,24,0,2,1.0,2,838723.0,2957.0,1.0,2.0
3,79.724550,92.698917,22.395136,8.059939,2.0,2.0,2.0,0,0.0,140,0,1,1.0,3,1267804.0,3037.0,2.0,3.0
4,101.541256,3.605055,81.449646,35.706541,3.0,2.0,2.0,0,0.0,159,2,1,1.0,4,1497166.0,2833.0,2.0,4.0


In [90]:
#fillna with mean
from sklearn.preprocessing import Imputer
def fill_missing(df):      
    imputer=Imputer(missing_values='NaN', strategy='mean', axis=0)
    imputer.fit(df)

    da=imputer.transform(df)
    col=df.columns

    df=pd.DataFrame(data=da,columns=col)
    return df

In [91]:
#fillna with mean
df_mean=fill_missing(df)

#fillna with interpolation
df_interpolate=df.interpolate()


In [92]:
df_mean.head(n=2)

,dst_dock,dst_mkt,dst_tower,dst_river,dining_rooms,bedrooms,bathrooms,king_visit,curse,king_blessing,farm_land,locations,holy_tree,builder,price,house_age,build_year,priced_year
0,138.600639,69.486467,14.394903,63.109038,3.0,4.0,2.0,0.0,0.0,160.0,0.0,0.0,1.0,0.0,1470912.0,2636.0,0.0,0.0
1,16.254871,34.145238,101.052598,53.249336,2.0,4.0,2.0,1.0,0.0,133.0,1.0,1.0,1.0,1.0,1241030.0,4270.0,1.0,1.0


In [93]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [19]:
d=df_interpolate

In [20]:
x=d.drop('price',axis=1)
y=d['price']



In [99]:

def crossvalid(df):
    x=df.drop('price',axis=1)
    y=df['price']
    
    X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.35,random_state=103)
    
    
    sc=StandardScaler()
    sc.fit(X_train)
    
    X_train=sc.transform(X_train)
    X_test=sc.transform(X_test)
    
    
    
    #rf=RandomForestRegressor(n_estimators=100)
    rf=MLPRegressor(shuffle=True,alpha=1,hidden_layer_sizes=(200, 200, 200, 200),tol=0,max_iter=500, verbose=True,random_state=1000)
    rf.fit(X_train,y_train)
    
    
    y_pred=rf.predict(X_test)
    
    r2s=metrics.r2_score(y_test,y_pred)
    
    test=pd.read_csv('csv/test.csv')

    test_house_id=test['house_id']

    
    test=filtering(test)
    
    #test=test.interpolate()
    test=fill_missing(test)
    


    pred_test=rf.predict(sc.transform(test))
    print("R2 score of test sample")
    print(r2s)
    
    
    
    id_vec=np.array(test_house_id) 
    sol_df=pd.DataFrame()
    sol_df['House ID']=id_vec
    sol_df['Golden Grains']=pred_test
   
    return sol_df
    
    
pass   
    

In [100]:
sol=crossvalid(df_mean)

Iteration 1, loss = 865294930145.67346191
Iteration 2, loss = 747995260679.70825195
Iteration 3, loss = 118624511548.25543213
Iteration 4, loss = 8558944383.58247375
Iteration 5, loss = 7481581710.76205444
Iteration 6, loss = 6749030160.67251873
Iteration 7, loss = 6155951485.28302860
Iteration 8, loss = 5664654303.89556122
Iteration 9, loss = 5243991733.38777828
Iteration 10, loss = 4880384872.10544586
Iteration 11, loss = 4536345040.10198021
Iteration 12, loss = 4246567203.23732281
Iteration 13, loss = 3964034856.06757355
Iteration 14, loss = 3711870992.84499645
Iteration 15, loss = 3487293662.10397577
Iteration 16, loss = 3272444413.78894281
Iteration 17, loss = 3074691056.08870888
Iteration 18, loss = 2898375993.12785482
Iteration 19, loss = 2731379479.34940004
Iteration 20, loss = 2575673373.98154497
Iteration 21, loss = 2450212443.65776873
Iteration 22, loss = 2317490288.79793692
Iteration 23, loss = 2200482936.17634869
Iteration 24, loss = 2092916725.09265852
Iteration 25, loss 

In [101]:
sol.head()

,House ID,Golden Grains
0,6e33d031,1.165321e+06
1,6e33d037,1.239065e+06
2,6e33d03d,1.491224e+06
3,6e33d043,1.221539e+06
4,6e33d045,1.339981e+06


In [98]:
sol.to_csv('submit_mean.csv',index=False)

In [140]:
rf=RandomForestRegressor()
rf.fit()

TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

In [30]:
test

,house_id,buid_date,date_priced,dst_dock,dst_mkt,dst_tower,dst_river,dining_rooms,bedrooms,bathrooms,king_visit,curse,king_blessing,farm_land,locations,holy_tree,builder,house_age,build_year,priced_year
0,6e33d031,1601-02-05 03:11:00,1610-03-11 04:59:00,105.449712,25.687517,124.656380,32.971288,3.0,2.0,3.0,0,0.0,96,0,0,1.0,0,3321.0,0.0,0.0
1,6e33d037,1605-02-17 14:36:00,1611-07-07 23:42:00,10.028074,NaN,148.906420,53.604283,2.0,4.0,2.0,0,0.0,143,1,1,1.0,1,2331.0,1.0,1.0
2,6e33d03d,1604-06-08 11:52:00,1607-08-01 19:40:00,46.697442,33.374570,118.027166,7.347228,NaN,3.0,3.0,0,0.0,151,0,2,1.0,2,1149.0,2.0,2.0
3,6e33d043,1600-03-13 22:21:00,1612-01-07 13:52:00,9.797009,9.624204,123.956946,24.799455,2.0,2.0,3.0,0,0.0,151,1,1,1.0,1,4316.0,3.0,3.0
4,6e33d045,1603-01-01 18:53:00,1612-01-17 05:27:00,51.371900,NaN,86.886693,5.552861,3.0,3.0,2.0,0,0.0,127,0,1,1.0,3,3302.0,4.0,3.0
5,6e33d049,1605-10-21 14:15:00,1612-05-14 19:22:00,29.824765,23.007202,63.369694,20.721687,4.0,2.0,2.0,0,1.0,108,0,0,0.0,1,2397.0,1.0,3.0
6,6e33d04d,1606-08-03 03:42:00,1608-11-02 00:50:00,31.737892,41.647926,76.754042,1.551319,2.0,2.0,2.0,0,0.0,113,0,1,1.0,1,821.0,5.0,4.0
7,6e33d051,1600-10-19 20:21:00,1612-10-19 07:59:00,13.943795,17.472071,54.192768,78.604543,3.0,2.0,4.0,0,0.0,94,0,0,1.0,4,4382.0,3.0,3.0
8,6e33d053,1602-10-13 11:14:00,1607-06-24 03:34:00,23.340074,144.284363,19.646201,36.137882,2.0,3.0,3.0,0,0.0,90,0,1,1.0,1,1714.0,6.0,2.0
9,6e33d055,1600-08-11 12:37:00,1608-06-06 16:22:00,24.864534,7.256853,87.784779,4.489005,2.0,3.0,3.0,0,0.0,78,0,1,1.0,1,2856.0,3.0,4.0


In [29]:
test=pd.read_csv('csv/test.csv')
test=filtering(test)

#test=test.interpolate()
test=fill_missing(test)


curse        0
holy_tree    0
dtype: int64
0
3500


TypeError: float() argument must be a string or a number, not 'datetime.datetime'

In [124]:
test=pd.read_csv('csv/test.csv')

test=filtering(test)

test=test.interpolate()

test_house_id=test['house_id']

test=drop_unused(test)

curse        0
holy_tree    0
dtype: int64
0
3500


In [128]:
id_vec=np.array(test_house_id) 

In [129]:
id_vec

array(['6e33d051', '6e33d0e0', '6e33d108', ..., '6e340598', '6e3405ec',
       '6e340674'], dtype=object)

In [130]:
sol_df=pd.DataFrame()
sol_df['id']=id_vec

In [132]:
sol_df

,id
0,6e33d051
1,6e33d0e0
2,6e33d108
3,6e33d16f
4,6e33d1b1
5,6e33d27e
6,6e33d2a1
7,6e33d34f
8,6e33d359
9,6e33d35f
